In [2]:
import math

In [3]:
def find_Hs(W, U, alpha0, A0, alphaZ, Az, d_h, d_r, d_z): 
    """
    Calculates the primary beam scattered from the primary barrier.

    Parameters:
    W (float): Workload (Gy/wk) for the worst-case Wall G.
    U (float): Usage factor for the worst-case Wall G.
    alpha0 (float): Albedo of the primary barrier, dependent on material, energy, 
                    angle of incidence, and reflection.
    A0 (float): Dependent on field size and d_h (m^2).
    alphaZ (float): Albedo of Wall Z, dependent on material, energy, 
                    angle of incidence, and reflection.
    Az (float): Area of the inner maze entrance as viewed from the center of the 
                primary barrier projected onto Wall Z (m^2).
    d_h (float): Distance from the source to the proximal surface of the primary barrier (m).
    d_r (float): Distance from the center of the primary barrier, past the edge of IMW, 
                 to point B (m).
    d_z (float): Distance from the point B to the plane of the door (m).
    """

    H_s = (W * U * alpha0 * A0 * alphaZ * Az) / (d_h * d_r * d_z)**2
    return H_s


def find_Hps(W, U, a, F, alpha1, A1, d_sca, d_sec, d_zz): 
    """

    Parameters:
    
    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    a : scatter fraction dependent on scatter angle and beam energy 
    F : field size (cm^2)
    alpha1 : albedo of A1. Dependent on material, 0.5 MeV, angle of incidence, angle of reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sca : distance from source to iso (m)
    d_sec : distance from iso to center of A1 (m)
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ps = (W * U * a * (F/400)* alpha1 * A1) / (d_sca * d_sec * d_zz)**2
    return H_ps


def find_Hls(W, U, C, alpha_ls, A1, d_sec, d_zz):
    """
    
    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    C : IMRT factor (unitless)
    alpha_ls : albedo of A1, dependent on material, Co-60
               angle of incidence, and reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sec : distance from iso to center of A1 (m) if gantry rotates uniformly
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ls = (10**-3 *W * U * C * alpha_ls * A1) / (d_sec * d_zz)**2
    return H_ls


def find_Hlt(W, C, U, t_slant, TVL1, TVLe, d_L):
    """

    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    C : IMRT factor (unitless)
    U : usage factor for worst case Wall G
    t_slant : slant thickness of IMW (cm)
    TVL1 : TVL of IMW material (cm). Dependent on incident energy
    TVLe : equilibrium TVL of IMW material (cm). Dependent on incident energy
    d_L : distance from source to center of plane of door (m)
    """

    H_lt = (10**-3 * W * C * U * 10**-(1 + (t_slant - TVL1) / TVLe)) / d_L**2
    return H_lt


In [4]:
x = find_Hs(2500,0.25,4*10**-3,4.41,20*10**-3,8.6666,5.25,6.01857,5.15555)
print('H_s')
print(f"{x*10**6} uSv/wk")

H_s
72.01154042910764 uSv/wk


In [5]:
y = find_Hps(2500, 0.25, 1.39*10**-3,1600, 22*10**-3,8.1428,1,6.9507,7.6)
print('H_ps')
print(f"{y*10**6} uSv/wk")

H_ps
223.08326114189148 uSv/wk


In [6]:
z = find_Hls(2500, 0.25, 3, 9*10**-3, 8.1428, 6.9507, 7.6)
print('H_ls')
print(f"{z*10**6} uSv/wk")

H_ls
49.24172703426317 uSv/wk


In [7]:
u = find_Hlt(2500, 3, 0.25, 86.9965, 34, 29, 5.981)
print('H_lt')
print(f"{u*10**6} uSv/wk")  

H_lt
77.98088326517457 uSv/wk


In [8]:
f = 0.25 # correction factor for 6-10 MV
H_G = f*x + y + z + u
print('H_G')
print(H_G*10**6)


H_G
368.30875654860614


In [9]:
H_tot = 2.64 * H_G * 10**6
print('H_tot')
print(f"{H_tot} uSv/wk")

H_tot
972.3351172883201 uSv/wk


In [41]:
P_controlled = 100 # uSv/wk
T = 1 # occupancy factor for treatment vault doors

B_needed = (P_controlled / T) / H_tot 
print('B_needed')
print(f"{B_needed}")

B_needed
0.10284520040671087


In [42]:
t_steel = 0.015875 # thickness of steel (cm)
TVL_steel_6MV = 10 # cm

TVL_Pb_6MV = 5.7 # cm 

B_steel = 10**-(t_steel / TVL_steel_6MV)
print('B_steel')
print(f"{B_steel}")


B_lead = B_needed / B_steel**2
print('B_lead')
print(f"{B_lead}")

# Find the thickness of lead needed to achieve the required barrier transmission
# using the TVL for lead at 6 MV
t_lead = TVL_Pb_6MV * -math.log10(B_lead)
print('t_lead')
print(f"{t_lead} cm")

B_steel
0.996351318837899
B_lead
0.10359982666716379
t_lead
5.612453535886364 cm


In [29]:
x2 = find_Hs(283.5,1,4*10**-3,4.41,20*10**-3,8.6666,5.25,6.01857,5.15555)
print('H_s')
print(f"{x2*10**6} uSv/wk")

H_s
32.664434738643244 uSv/wk


In [32]:
y2 = find_Hps(283.5,1,8.24*10**-3,32400,22*10**-3,8.14285,4.5,5.5509008,7.6)
print('H_ps')
print(f"{y2*10**6} uSv/wk")

H_ps
940.55731397177 uSv/wk


In [36]:
z2 = find_Hls(283.5,1,1,9*10**-3,8.14285,7.603453163,7.6)
print('H_ls')
print(f"{z2*10**6} uSv/wk")

H_ls
6.221900658622358 uSv/wk


In [43]:
v2 = find_Hlt(283.5,1,1,86.9965,34,29,5.981)
print('H_lt')
print(f"{v2*10**6} uSv/wk")

H_lt
11.790709549694396 uSv/wk
